In [1]:
param_market = 'US'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script></body></html>')[0] + ']}]}')
                
                # = = = = = = = = = = = = = = =
                
                market_id = json_data['o']['g']['marketplaceId'].strip()
                
                # = = = = = = = = = = = = = = =
                
                country = json_data['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = json_data['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = json_data['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = json_data['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US':
                    if market_id != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                else:
                    pass #
                    
                # = = = = = = = = = = = = = = =
                
                specification = json_data['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US' or param_market == 'AU':
                    if 'VEHICLE_SELECTION' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                elif param_market == 'DE':
                    if 'MOTORS_TAB' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                        
                # = = = = = = = = = = = = = = =
                
                title = json_data['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in json_data['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = json_data['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'binModel' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in json_data['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = json_data['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if param_market == 'US' or param_market == 'AU':
                            if 'available' in text['text']:
                                available = text['text'].strip()
                            elif 'sold' in text['text']:
                                sold = text['text'].strip()
                        elif param_market == 'DE':
                            if 'verfügbar' in text['text']:
                                available = text['text'].strip()
                            elif 'verkauft' in text['text']:
                                sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Item_Number': input_.loc[a, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Description_Url': description_url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                dict_name = {}
                if param_market == 'US' or param_market == 'AU':
                    condition = 'Condition'
                elif param_market == 'DE':
                    condition = 'Artikelzustand'

                for i, name_value in zip(range(len(specification)), specification):
                    name = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if i == 0 and name == condition:
                        df_temp[condition] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                    else:
                        name = name.lower()
                        if name in dict_name:
                            dict_name[name] += 1
                        else:
                            dict_name[name] = 1
                        df_temp[name if dict_name[name] == 1 else name + '-' + str(dict_name[name])] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Item_Number': input_.loc[a, 'Item_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./item_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：5656

[ok] - 284240322618
[尝试次数：1] - [剩余数量：5636] - [当前时间：18:30:14]

[ok] - 282866092544
[尝试次数：1] - [剩余数量：5635] - [当前时间：18:30:14]

[ok] - 284232775523
[尝试次数：1] - [剩余数量：5634] - [当前时间：18:30:15]

[ok] - 273977223636
[尝试次数：1] - [剩余数量：5633] - [当前时间：18:30:15]

[ok] - 303939114015
[尝试次数：1] - [剩余数量：5632] - [当前时间：18:30:15]

[ok] - 284240562623
[尝试次数：1] - [剩余数量：5631] - [当前时间：18:30:15]

[ok] - 284232789435
[尝试次数：1] - [剩余数量：5630] - [当前时间：18:30:15]

[ok] - 302654932487
[尝试次数：1] - [剩余数量：5629] - [当前时间：18:30:15]

[ok] - 303942818317
[尝试次数：1] - [剩余数量：5628] - [当前时间：18:30:16]

[ok] - 274732328540
[尝试次数：1] - [剩余数量：5627] - [当前时间：18:30:16]

[ok] - 284232798953
[尝试次数：1] - [剩余数量：5626] - [当前时间：18:30:16]

[ok] - 284238171734
[尝试次数：1] - [剩余数量：5625] - [当前时间：18:30:16]

[ok] - 284240396275
[尝试次数：1] - [剩余数量：5624] - [当前时间：18:30:16]

[ok] - 303393671090
[尝试次数：1] - [剩余数量：5623] - [当前时间：18:30:16]

[ok] - 303941511458
[尝试次数：1] - [剩余数量：5622] - [当前时间：18:30:16]

[ok] - 273163050275
[尝试次数：1] - [剩余数量：5621] - [当前时间：18:30:18]

[ok] - 274115621686
[尝试次数：4] - [剩余数量：5504] - [当前时间：18:30:44]

[ok] - 303346721738
[尝试次数：1] - [剩余数量：5503] - [当前时间：18:30:44]

[ok] - 285411478772
[尝试次数：1] - [剩余数量：5502] - [当前时间：18:30:44]



KeyboardInterrupt
2024-02-29T10:30:45Z


KeyboardInterrupt: 

[ok] - 303273845056
[尝试次数：1] - [剩余数量：5501] - [当前时间：18:30:45]

